In [1]:
import os
import geopandas as gpd
import pandas as pd

In [10]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

In [3]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [7]:
enriched_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0"

In [8]:
enriched_list = os.listdir(enriched_path)

In [9]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [3]:
report_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb'

In [11]:
# skip the append, reports gdb file path
skip_list = ['appended.gdb', 'reports.gdb']

In [12]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    if f in skip_list:
        continue
    f_path = enriched_path + r"\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [10]:
# one time change for activity category

In [21]:
from utils.category import categorize_activity
from utils.standardize_domains import standardize_domains
from utils.counts_to_mas import counts_to_mas

In [29]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-07-21 13:53:43,577 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-07-21 13:53:43,577 INFO  [process.append_polygon]  Concatenate all polygon records
2025-07-21 13:53:43,578 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250710' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-21 13:53:43,683 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'CNRA_enriched_20250626_polygon' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CNRA_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-21 13:53:47,823 INF

In [30]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [31]:
# grab timber non spatial path again
timber_nonspatial_path = None
for p in enriched_layers['point']:
    if 'Timber_Nonspatial' in p['gdb_path']:
        timber_nonspatial_path = p
        break

In [32]:
timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [33]:
%%time
for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    # drop unwanted artifact columns from California boundary df
    append_clipped = append_clipped.drop(['index_right', 'Shape_Area', 'Shape_Length'], axis=1)
    
    # industry nonspatial is by design out of california bounds and got clipped, manually concat it back
    if lyr_name == 'appended_point':
        append_clipped = pd.concat([append_clipped, timber_nonspatial], ignore_index=True)
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-07-21 14:00:50,749 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-21 14:00:50,749 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-21 14:00:50,778 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-21 14:01:21,262 INFO  [pyogrio._io           ]  Created 455,318 records
2025-07-21 14:01:28,524 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_poly
2025-07-21 14:03:13,678 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-21 14:03:13,678 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-21 14:03:13,687 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-21 14:03:34,199 INFO  [pyogrio._io           ]  Created 131,658 records
2025-07-21 14:03:38,200 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_line
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-21 14:03:41,240 INFO  [pyogrio._io           ]  Created 20,443 records
2025-07-21 14:03:41,387 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_point


CPU times: total: 51min 20s
Wall time: 9min 15s


In [34]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [35]:
gpd.list_layers(append_path)

,name,geometry_type
0,appended_poly,MultiPolygon
1,appended_line,MultiLineString
2,appended_point,Point


In [36]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [37]:
enriched_points[(enriched_points.AGENCY =='<NA>')].PROJECTID_USER.apply(lambda x: x[:5]).unique()

array([], dtype=object)

In [38]:
enriched_polygons[(enriched_polygons.COUNTS_TO_MAS=="YES") & (enriched_polygons.ADMINISTERING_ORG.isna())]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry


In [39]:
#excel_out = pd.concat([enriched_points, enriched_lines, enriched_polygons])
#excel_out.drop(['geometry'], axis=1).to_csv('its_all.csv')

In [40]:
from datetime import datetime

In [41]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'DOE', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [42]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [15]:
pfirs_path = {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\PFIRS_1950_2025.gdb',
  'layer_name': 'PFIRS_20250718'}

In [16]:
pfirs_gdf = gpd.read_file(pfirs_path['gdb_path'], driver='OpenFileGDB', layer=pfirs_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [27]:
out = standardize_domains(categorize_activity(pfirs_gdf)), 1950, 2025

In [29]:
out[0]

array(['PRESCRB_FIRE'], dtype=object)

In [43]:
save_gdf_to_gdb(activity_report_gdf, 
                report_path, 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-07-21 14:04:30,028 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-21 14:04:30,029 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-21 14:04:30,068 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb


2025-07-21 14:04:37,689 INFO  [pyogrio._io           ]  Created 498,157 records
2025-07-21 14:04:40,657 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb activity_report20250721


In [4]:
activity_report_gdf = gpd.read_file(report_path, driver='OpenFileGDB', layer='activity_report20250721')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [31]:
activity_report_gdf.ACTIVITY_STATUS.unique()

array(['COMPLETE', 'ACTIVE'], dtype=object)

In [6]:
activity_report_gdf[activity_report_gdf.ACTIVITY_CAT.isna()]

,AGENCY,ADMINISTERING_ORG,PRIMARY_OWNERSHIP_GROUP,COUNTY,REGION,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_END,Year_txt,ENTITY_TYPE,geometry
172731,NPS,NPS,FEDERAL,SHA,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,COMPLETE,560.0,AC,2002-11-04 00:00:00+00:00,2002,Federal,POINT (-121.45 40.548)
172732,NPS,NPS,FEDERAL,MRN,CENTRAL_COAST,BROADCAST_BURN,None,FOREST,COMPLETE,200.0,AC,2002-10-25 00:00:00+00:00,2002,Federal,POINT (-122.911 38.085)
172733,NPS,NPS,FEDERAL,HUM,NORTH_COAST,BROADCAST_BURN,None,FOREST,COMPLETE,200.0,AC,2003-11-01 00:00:00+00:00,2003,Federal,POINT (-123.87 41.137)
172734,NPS,NPS,FEDERAL,FRE,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,COMPLETE,1.0,AC,2002-10-05 00:00:00+00:00,2002,Federal,POINT (-118.632 36.789)
172735,NPS,NPS,FEDERAL,FRE,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,COMPLETE,155.0,AC,2006-11-03 00:00:00+00:00,2006,Federal,POINT (-118.947 36.748)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173528,NPS,NPS,FEDERAL,HUM,NORTH_COAST,BROADCAST_BURN,None,FOREST,COMPLETE,854.0,AC,2022-10-08 00:00:00+00:00,2022,Federal,POINT (-123.874 41.138)
173532,NPS,NPS,FEDERAL,SHA,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,COMPLETE,225.0,AC,2023-09-27 00:00:00+00:00,2023,Federal,POINT (-121.568 40.524)
173533,NPS,NPS,FEDERAL,MPA,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,COMPLETE,202.0,AC,2023-05-02 00:00:00+00:00,2023,Federal,POINT (-119.633 37.533)
173534,NPS,NPS,FEDERAL,MPA,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,ACTIVE,20.0,AC,2023-06-13 00:00:00+00:00,2023,Federal,POINT (-119.61 37.508)


In [32]:
activity_report_gdf[activity_report_gdf.Year_txt == '2023'].groupby(['AGENCY', 'ADMINISTERING_ORG']).sum('ACTIVITY_QUANTITY')

ACTIVITY_QUANTITY
AGENCY ADMINISTERING_ORG                   
CALSTA CALTRANS               135940.980700
CNRA   CALFIRE                183216.092500
       CCC                      1026.002762
       CDFW                   117347.586539
       PARKS                   12055.327324
       RMC                       369.130000
       SCC                       814.500000
       SDRC                     1903.757000
       SMMC                      426.889317
       SNC                      4380.430000
       TAHOE                     302.200000
       WCB                      5016.840000
DOD    DOD                      1883.230000
DOE    DOE                      7186.925000
DOI    BIA                      8636.000000
       BLM                     27278.800174
       FWS                     95887.000000
NPS    NPS                      5285.000000
TIMBER TIMBER                 234471.832990
USDA   USFS                   297118.800000

In [49]:
activity_report_gdf[activity_report_gdf.Year_txt == '2022'].groupby(['AGENCY', 'ADMINISTERING_ORG']).sum('ACTIVITY_QUANTITY')

ACTIVITY_QUANTITY
AGENCY ADMINISTERING_ORG                   
CALSTA CALTRANS               122514.696340
CNRA   CALFIRE                160727.116890
       CCC                      1117.083820
       CDFW                   108346.846762
       PARKS                   12814.796613
       RMC                        79.470000
       SCC                       891.150000
       SDRC                      317.920000
       SMMC                      499.324782
       TAHOE                     310.700000
       WCB                      3509.710000
DOD    DOD                      4667.000000
DOE    DOE                      4412.690000
DOI    BIA                      7401.000000
       BLM                     27152.304505
       FWS                     52991.000000
NPS    NPS                      3609.000000
TIMBER TIMBER                 222278.078178
USDA   USFS                   142327.600000

In [30]:
activity_report_gdf[(activity_report_gdf.ACTIVITY_DESCRIPTION == "AMW_AREA_RESTOR") | (activity_report_gdf.ACTIVITY_DESCRIPTION == "OAK_WDLND_MGMT")]

,AGENCY,ADMINISTERING_ORG,PRIMARY_OWNERSHIP_GROUP,COUNTY,REGION,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_END,Year_txt,geometry,ENTITY_TYPE
131462,CNRA,CDFW,STATE,MAD,SIERRA_NEVADA,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,3.680000,AC,2022-01-31 00:00:00+00:00,2022,POINT (56710.253 -88969.102),State
20390,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,147.163751,AC,2021-12-31 00:00:00+00:00,2021,POINT (-284207.678 -48707.918),Timber Companies
20391,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,280.000000,AC,2022-12-31 00:00:00+00:00,2022,POINT (-284207.678 -48707.918),Timber Companies
20392,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,14.327502,AC,2023-12-31 00:00:00+00:00,2023,POINT (-284207.678 -48707.918),Timber Companies
20393,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,0.000000,AC,2024-12-31 00:00:00+00:00,2024,POINT (-284207.678 -48707.918),Timber Companies
4574,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,5.170592,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.03 40.335),State
4575,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,0.241517,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.034 40.337),State
4577,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,4.860597,AC,2023-08-25 00:00:00+00:00,2023,POINT (-124.039 40.34),State
4580,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,1.676779,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.032 40.336),State
5427,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,2.321638,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.033 40.337),State
